# AutoGen RoundRobinGroupChat
A team is a group of agents that work together to achieve a common goal.


## Load Azure Configurations

In [27]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [28]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

azure_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

In [29]:
import arxiv

def search_arxiv(topic: str, max_results: int = 5):
    client = arxiv.Client()

    # Search for the latest articles matching the queried topic
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = client.results(search)

    return list(papers)

a = search_arxiv("machine learning", max_results=3)

In [30]:
[paper.entry_id.split('/')[-1] for paper in a] 

['2506.19852v1', '2506.19850v1', '2506.19847v1']

In [31]:
a.sort(key=lambda x: x.published, reverse=True)

In [32]:
[str(paper.published.date()) for paper in a]

['2025-06-24', '2025-06-24', '2025-06-24']

In [33]:
[paper.pdf_url for paper in a]

['http://arxiv.org/pdf/2506.19852v1',
 'http://arxiv.org/pdf/2506.19850v1',
 'http://arxiv.org/pdf/2506.19847v1']

In [34]:
[author.name for paper in a for author in paper.authors]

['Xingyang Li',
 'Muyang Li',
 'Tianle Cai',
 'Haocheng Xi',
 'Shuo Yang',
 'Yujun Lin',
 'Lvmin Zhang',
 'Songlin Yang',
 'Jinbo Hu',
 'Kelly Peng',
 'Maneesh Agrawala',
 'Ion Stoica',
 'Kurt Keutzer',
 'Song Han',
 'Yuqi Wang',
 'Xinghang Li',
 'Wenxuan Wang',
 'Junbo Zhang',
 'Yingyan Li',
 'Yuntao Chen',
 'Xinlong Wang',
 'Zhaoxiang Zhang',
 'Zeju Qiu',
 'Weiyang Liu',
 'Adrian Weller',
 'Bernhard Schölkopf']

## Creating a Team

RoundRobinGroupChat is a team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We create a team with two AssistantAgent and a TextMentionTermination condition that stops the team when a specific word is detected in the agent’s response.

In [35]:
import arxiv
import json
import os

from langchain_community.document_loaders import PyPDFLoader
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import SourceMatchTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

PAPER_DIR = "../ArxivResearcher/papers"

# Create the user proxy agent.
user_proxy_agent = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Use arxiv to find the papers 
def search_arxiv(topic: str, max_results: int = 5):
    client = arxiv.Client()

    # Search for the latest articles matching the queried topic
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending
    )

    papers = client.results(search)
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
            return file_path  # If file exists, return the path without updating
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_id = paper.entry_id.split('/')[-1] # Use the last part of the entry_id as a unique identifier
        paper_ids.append(paper_id)
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper_id] = paper_info

    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json_file.write(json.dumps(papers_info, indent=2))
    print(f"Papers metadata saved to {file_path}")

    return file_path

# Create the arxiv search agent.
arxiv_search_agent = AssistantAgent(
    name="arxiv_search",
    description="An agent that can retrieve research papers related to user input on arXiv \
        and save the metadata of the papers in a JSON file.",
    model_client=azure_model_client,
    tools=[search_arxiv],
    system_message="You are an expert academic search assistant. \
        Your task is to find the most relevant and recent papers on arXiv \
        for given user input, henceforth called topic, and stores them in JSON file 'papers_info.json'."
)
# Create the content extractor agent that downloads the PDFs and extracts the content.
def content_extractor(file_path: str):

    with open(file_path, "r") as json_file:
        papers_info = json.load(json_file)

    papers_contents = {}
    # Iterate through each paper's info and download the PDF
    for paper_id, info in papers_info.items():
        pdf_path = os.path.join(os.path.dirname(file_path), f"{paper_id}.pdf")
        
        # Download the PDF if it doesn't exist
        if not os.path.exists(pdf_path):
            paper = next(arxiv.Client().results(arxiv.Search(id_list=[paper_id])))
            paper.download_pdf(filename=pdf_path)

        # Load the PDF and extract text
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()

        # Extract the raw text from the document
        if documents:
            papers_contents[paper_id] = documents[0].page_content[:5000]  # Limit to first 5000 characters for brevity
        else:
            papers_contents[paper_id] = "No content found."

    # Save the extracted contents to a JSON file
    contents_file_path = os.path.join(os.path.dirname(file_path), "papers_contents.json")
    with open(contents_file_path, "w") as contents_file:
        json.dump(papers_contents, contents_file, indent=2)

    return contents_file_path

# Create the Content Extractor agent.
info_extractor_agent = AssistantAgent(
    name="content_extractor",
    description="An agent that extracts raw text from all the downloaded arxiv research papers \
    obtained from the arxiv search agent.",
    model_client=azure_model_client,
    tools=[content_extractor],
    system_message="You are a research papers content extractor. \
        Read paper_id from papers_info.json. Download PDFs and extract the raw text content from each paper. \
        Ignore the papers whose content could not be extracted.\
        The extracted contents should be saved in a JSON file named 'papers_contents.json'.\
        Return the path to the JSON file."
)

# Function to generate full text from the extracted papers contents.
def generate_full_text(content_file_path: str):
    """Combine the raw texts of the papers from the JSON file."""
    with open(content_file_path, "r") as json_file:
        papers_contents = json.load(json_file)
    full_text = "\n".join(papers_contents.values())
    print(f"Read {len(papers_contents)} papers contents from {content_file_path}.")
    print(f"Total characters in papers contents: {len(full_text)}")
    
    return full_text

# Create the Research Summarizer agent.
research_summarizer_agent = AssistantAgent(
    name="research_summarizer",
    description="An agent that writes a single coherent summary on a topic from the top 5 research papers on arXiv.",
    model_client=azure_model_client,
    tools=[generate_full_text], 
    system_message="You are an expert academic researcher. \
        Your task is to write a single coherent summary relevant to the given topic in under 5000 words. \
        To do this, first generate full text from the contents of all papers by reading \
        the JSON file 'papers_contents.json' located in the topic's subdirectory inside the project directory. \
        The file contains the paper_ids and raw text content from each paper. \
        Focus on common themes and significant advancements. \
        Do not just summarize the papers individually, Always address the topic at hand."   
)

# Create the summary writer agent that writes the generated summary to a text file.
def write_summary_to_file(summary: str, topic: str):
    """Write the generated summary to a text file named 'summary.txt' in the topic's subdirectory."""
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    summary_file_path = os.path.join(path, "summary.txt")
    
    with open(summary_file_path, "w") as summary_file:
        summary_file.write(summary)
    
    print(f"Summary written to {summary_file_path}")
    return summary_file_path

summary_writer_agent = AssistantAgent(
    name="summary_writer",
    description="An agent that writes the generated summary to a text file.",
    model_client=azure_model_client,
    tools=[write_summary_to_file],
    system_message="You are a summary writer. \
        Your task is to write the given summary to a text file named 'summary.txt' in the topic's subdirectory."
)

## Define a Termination Condition And A Round Robin Group Chat Team

In [36]:
text_mention_termination = TextMentionTermination("QUIT")
source_match_termination = SourceMatchTermination('summary_writer')
termination = text_mention_termination | source_match_termination

In [37]:
# Create a RoundRobin team with the all the agents defined above.
team = RoundRobinGroupChat(
    [user_proxy_agent, arxiv_search_agent, info_extractor_agent, \
     research_summarizer_agent, summary_writer_agent],
    termination_condition=termination
)

In [38]:
# Stream the messages to the console.
await Console(team.run_stream())  
#await team.reset()  # Reset the team for the next run.

---------- TextMessage (user_proxy) ----------
LoRA
---------- ToolCallRequestEvent (arxiv_search) ----------
[FunctionCall(id='call_ZTd7bkyfAQ2BWTnmR1sPfiEh', arguments='{"topic":"LoRA","max_results":5}', name='search_arxiv')]
---------- ToolCallExecutionEvent (arxiv_search) ----------
[FunctionExecutionResult(content='../ArxivResearcher/papers/lora/papers_info.json', name='search_arxiv', call_id='call_ZTd7bkyfAQ2BWTnmR1sPfiEh', is_error=False)]
---------- ToolCallSummaryMessage (arxiv_search) ----------
../ArxivResearcher/papers/lora/papers_info.json
---------- ToolCallRequestEvent (content_extractor) ----------
[FunctionCall(id='call_tu72wcxSynmMpHX4AZ6MFJL4', arguments='{"file_path":"../ArxivResearcher/papers/lora/papers_info.json"}', name='content_extractor')]
---------- ToolCallExecutionEvent (content_extractor) ----------
[FunctionExecutionResult(content='../ArxivResearcher/papers/lora/papers_contents.json', name='content_extractor', call_id='call_tu72wcxSynmMpHX4AZ6MFJL4', is_e

TaskResult(messages=[UserInputRequestedEvent(source='user_proxy', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 25, 20, 19, 39, 111588, tzinfo=datetime.timezone.utc), request_id='00b52541-0571-47cf-9fc0-6ef6a67457df', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 25, 20, 19, 44, 286630, tzinfo=datetime.timezone.utc), content='LoRA', type='TextMessage'), ToolCallRequestEvent(source='arxiv_search', models_usage=RequestUsage(prompt_tokens=112, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 6, 25, 20, 19, 45, 733456, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_ZTd7bkyfAQ2BWTnmR1sPfiEh', arguments='{"topic":"LoRA","max_results":5}', name='search_arxiv')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='arxiv_search', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 25, 20, 19, 47, 67231, tzin

## Aborting a Team
You can abort a call to run() or run_stream() during execution by setting a CancellationToken passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [39]:
import asyncio
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.
